In [1]:
# Install required packages
!pip install wget
!pip install open3d
!pip install ultralytics
!pip install matplotlib
!pip install deep-sort-realtime
!pip install torchreid

  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9656 sha256=01c67d3e0905b4671c37e369d774ca2a5b5da56f3557aa3899941a4d81441cdd
  Stored in directory: /root/.cache/pip/wheels/40/b3/0f/a40dbd1c6861731779f62cc4babcb234387e11d697df70ee97
Successfully built wget
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.7/447.7 MB 3.5 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 97.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.0/228.0 kB 16.2 MB/s eta 0:00:00
  Attempting uninstall: werkzeug
    Found existing installation: Werkzeug 3.1.3
    Uninstalling Werkzeug-3.1.3:
      Successfully uninstalled Werkzeug-3.1.3
  Attempting uninstall: flask
    Found existing installation: Flask 3.1.0
    Uninstalling Flask-3.1.0:
      Successfully uninstalled Flask-3.1.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import os
import sys
import cv2
import numpy as np
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt
from scipy.stats import multivariate_normal
import torch


In [3]:
# Clone UFLD repository
os.system("git clone https://github.com/ibaiGorordo/Ultrafast-Lane-Detection-Inference-Pytorch-.git")
print("✅ Cloned Ultrafast-Lane-Detection repository")

Cloning into 'Ultrafast-Lane-Detection-Inference-Pytorch-'...


✅ Cloned Ultrafast-Lane-Detection repository


In [23]:
# Define paths
input_video_path = "/kaggle/input/test-example/example.mp4"
output_video_path = "/kaggle/working/output_video.mp4"
mean_future_output_path = "/kaggle/working/mean_future_output.mp4"
dist_future_output_path = "/kaggle/working/dist_future_output.mp4"
ufld_model_path = "/kaggle/input/culane/pytorch/default/1/culane_18.pth"
repo_path = "/kaggle/working/Ultrafast-Lane-Detection-Inference-Pytorch-/ultrafastLaneDetector"
bev_points_csv = "/kaggle/working/bev_points.csv"

In [24]:
# Define paths
detector_path = os.path.join(repo_path, "ultrafastLaneDetector")
model_path = os.path.join(detector_path, "model")
simple_file_path = os.path.join(detector_path, "ultrafastLaneDetector.py")
model_file_path = os.path.join(model_path, "model.py")

In [25]:
# Add repo path to sys.path
sys.path.append(repo_path)
try:
    from ultrafastLaneDetector import UltrafastLaneDetector, ModelType
    from ultrafastLaneDetector.model import parsingNet
    print("✅ Successfully imported UltrafastLaneDetector and parsingNet")
except Exception as e:
    print(f"❌ Import failed: {e}")

❌ Import failed: No module named 'ultrafastLaneDetector.model'; 'ultrafastLaneDetector' is not a package


In [ ]:
# Add repo path to sys.path
sys.path.append(repo_path)

# Load UFLD model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if not os.path.exists(ufld_model_path):
    raise FileNotFoundError(f"❌ culane_18.pth not found at {ufld_model_path}")
try:
    lane_detector = UltrafastLaneDetector(ufld_model_path, model_type=ModelType.CULANE, use_gpu=(device.type == "cuda"))
    lane_detector.model.to(device)
    print("✅ UFLD Model loaded")
except Exception as e:
    print(f"❌ Failed to load UFLD model: {e}")
    lane_detector = None

In [ ]:
# Load YOLO model
try:
    yolo_model = YOLO('yolov8m.pt')
    print("✅ YOLO Model loaded")
except Exception as e:
    print(f"❌ Failed to load YOLO model: {e}")
    yolo_model = None


In [ ]:
# Initialize DeepSort
deep_sort = DeepSort()

In [ ]:
# Initialize video capture
cap = cv2.VideoCapture(input_video_path)
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))
mean_out = cv2.VideoWriter(mean_future_output_path, fourcc, fps, (1000, 1000))
dist_out = cv2.VideoWriter(dist_future_output_path, fourcc, fps, (1000, 1000))

if not cap.isOpened():
    print(f"❌ Cannot open video: {input_video_path}")
    exit()

print("✅ Processing video...")


In [ ]:

# Class colors and names
CLASS_COLORS = {
    0: (0, 0, 255),      # person
    1: (0, 255, 0),      # bicycle
    2: (255, 255, 255),  # car
    3: (255, 255, 0),    # motorcycle
    5: (255, 0, 255),    # bus
    7: (0, 255, 255),    # truck
    9: (0, 255, 0)       # traffic light
}
CLASS_NAMES = yolo_model.model.names
SHOW_INFO_CLASSES = {0, 1, 2, 3, 5, 7}

# Camera parameters (placeholders)
focal_length = 1000
cx = frame_width // 2
cy = frame_height // 2
# Simulated offsets for front left and front right in BEV space (meters)
offset_front_left = -10
offset_front_right = 10
# Simulated FOV for transformations (degrees)
fov_simulated = 60

# Distance Estimator
def estimate_distance(bbox, focal_length=720, default_height=1.6):
    x, y, w, h = bbox
    if h <= 0:
        return 999
    return round((default_height * focal_length) / h, 1)

# Lane Assignment
def assign_lane(x_center, lane_boundaries):
    for i in range(len(lane_boundaries) - 1):
        if lane_boundaries[i + 1] <= x_center <= lane_boundaries[i]:
            return str(i + 1)
    return "_"

# Function to convert 2D lane points to 3D with offset
def compute_lane_points_3d(lanes_points, depth_map, focal_length, cx, cy, orig_shape, model_shape, x_offset=0):
    orig_width, orig_height = orig_shape
    model_width, model_height = model_shape
    depth_map = cv2.resize(depth_map, (model_width, model_height), interpolation=cv2.INTER_LINEAR)
    lane_points_3d = []
    
    for lane in lanes_points:
        lane_3d = []
        for point in lane:
            u, v = int(point[0]), int(point[1])
            z = 10.0  # Fixed depth
            x = (u - cx) * z / focal_length + x_offset
            y = (v - cy) * z / focal_length
            x = np.clip(x, -50, 50)
            y = np.clip(y, -50, 50)
            z = np.clip(z, 0, 100)
            lane_3d.append([x, y, z])
        lane_points_3d.append(lane_3d)
    
    return lane_points_3d


In [ ]:

# Function to transform object positions based on simulated view
def transform_object_position(obj_center, view_type, x_offset=0, fov=60):
    x, y, z = obj_center
    if view_type == "front":
        return np.array([x, y, z])
    elif view_type == "front_left":
        angle = np.radians(fov / 2)
        x_new = x * np.cos(angle) - z * np.sin(angle) + x_offset
        z_new = x * np.sin(angle) + z * np.cos(angle)
        return np.array([x_new, y, z_new])
    elif view_type == "front_right":
        angle = np.radians(-fov / aky2)
        x_new = x * np.cos(angle) - z * np.sin(angle) + x_offset
        z_new = x * np.sin(angle) + z * np.cos(angle)
        return np.array([x_new, y, z_new])
    return np.array([x, y, z])

# Simple trajectory prediction
def predict_trajectory(obj, time_horizon=3.0, dt=0.1):
    x, z = obj["center"][0], obj["center"][2]
    speed = 10.0
    vx, vz = 0.0, speed
    steps = int(time_horizon / dt)
    x_future = x + vx * time_horizon
    z_future = z + vz * time_horizon
    mean = [x_future, z_future]
    cov = [[2.0, 0.0], [0.0, 5.0]]
    future_dist = multivariate_normal(mean=mean, cov=cov)
    return np.array([x_future, z_future]), future_dist

# BEV visualization
def create_birds_eye_view(detected_objects, lane_points_3d_all, grid_size=200, frame_idx=0):
    x_range = (-50, 50)
    y_range = (0, 100)
    lane_width = 3.5
    
    mean_futures = []
    future_dists = []
    for obj in detected_objects:
        mean_future, future_dist = predict_trajectory(obj)
        mean_futures.append(mean_future)
        future_dists.append(future_dist)
        print(f"Object {obj['type']} predicted future position: ({mean_future[0]:.2f}, {mean_future[1]:.2f})")
    
    # Mean Future Plot
    plt.figure(figsize=(10, 10), dpi=100)
    plt.xlim(x_range[0], x_range[1])
    plt.ylim(y_range[0], y_range[1])
    
    # Draw road background
    plt.fill_between(x_range, 0, 100, color='gray', alpha=0.3)
    
    # Draw lanes
    for lane_points_3d in lane_points_3d_all:
        for lane in lane_points_3d:
            if len(lane) > 0:
                lane = np.array(lane)
                lane_x, lane_z = lane[:, 0], lane[:, 2]
                plt.fill_betweenx(lane_z, lane_x - lane_width/2, lane_x + lane_width/2, color='white', alpha=0.8)
                plt.plot(lane_x, lane_z, color='black', linewidth=1, linestyle='--')
    
    # Draw ego vehicle
    ego_x, ego_z = 0, 0
    plt.gca().add_patch(plt.Rectangle((ego_x-1.5, ego_z-2), 3, 4, color='blue', alpha=0.7))
    plt.text(ego_x, ego_z+3, "Ego", color='black', fontsize=8)
    
    # Draw objects
    for obj, mean_future in zip(detected_objects, mean_futures):
        x, z = mean_future
        color = [c/255 for c in obj["color"]]
        rect = plt.Rectangle((x-1, z-2), 2, 4, color=color, alpha=0.7)
        plt.gca().add_patch(rect)
        plt.text(x, z+3, f"{obj['type']} ({obj['confidence']*100:.1f}%)", color='black', fontsize=8)
    
    plt.title("Mean Future")
    plt.xlabel("X (meters)")
    plt.ylabel("Z (meters)")
    
    plt.gcf().canvas.draw()
    mean_img = np.frombuffer(plt.gcf().canvas.tostring_rgb(), dtype=np.uint8)
    mean_img = mean_img.reshape(plt.gcf().canvas.get_width_height()[::-1] + (3,))
    print(f"Mean image shape: {mean_img.shape}")
    
    if frame_idx == 5:
        cv2.imwrite("/kaggle/working/mean_future_frame5.jpg", cv2.cvtColor(mean_img, cv2.COLOR_RGB2BGR))
        print("✅ Saved Mean Future plot for frame 5 to /kaggle/working/mean_future_frame5.jpg")
    
    plt.close()
    
    # Distribution of Futures Plot
    plt.figure(figsize=(10, 10), dpi=100)
    plt.xlim(x_range[0], x_range[1])
    plt.ylim(y_range[0], y_range[1])
    
    # Draw road background
    plt.fill_between(x_range, 0, 100, color='gray', alpha=0.3)
    
    # Draw lanes
    for lane_points_3d in lane_points_3d_all:
        for lane in lane_points_3d:
            if len(lane) > 0:
                lane = np.array(lane)
                lane_x, lane_z = lane[:, 0], lane[:, 2]
                plt.fill_betweenx(lane_z, lane_x - lane_width/2, lane_x + lane_width/2, color='white', alpha=0.8)
                plt.plot(lane_x, lane_z, color='black', linewidth=1, linestyle='--')
    
    # Draw ego vehicle
    plt.gca().add_patch(plt.Rectangle((ego_x-1.5, ego_z-2), 3, 4, color='blue', alpha=0.7))
    plt.text(ego_x, ego_z+3, "Ego", color='black', fontsize=8)
    
    # Draw heatmap
    x_grid, z_grid = np.meshgrid(
        np.linspace(x_range[0], x_range[1], 200),
        np.linspace(y_range[0], y_range[1], 200)
    )
    pos = np.dstack((x_grid, z_grid))
    heatmap = np.zeros_like(x_grid)
    for future_dist in future_dists:
        heatmap += future_dist.pdf(pos)
    
    if heatmap.max() > 0:
        heatmap = heatmap / heatmap.max()
    
    plt.imshow(heatmap, extent=(x_range[0], x_range[1], y_range[0], y_range[1]), 
               origin='lower', cmap='coolwarm', alpha=0.6)
    plt.colorbar(label='Probability')
    
    plt.title("Distribution of Futures")
    plt.xlabel("X (meters)")
    plt.ylabel("Z (meters)")
    
    plt.gcf().canvas.draw()
    dist_img = np.frombuffer(plt.gcf().canvas.tostring_rgb(), dtype=np.uint8)
    dist_img = dist_img.reshape(plt.gcf().canvas.get_width_height()[::-1] + (3,))
    print(f"Distribution image shape: {dist_img.shape}")
    
    if frame_idx == 5:
        cv2.imwrite("/kaggle/working/dist_future_frame5.jpg", cv2.cvtColor(dist_img, cv2.COLOR_RGB2BGR))
        print("✅ Saved Distribution of Futures plot for frame 5 to /kaggle/working/dist_future_frame5.jpg")
    
    plt.close()
    
    return mean_img, dist_img

In [ ]:

# Video processing loop
frame_idx = 0
bev_points_data = []
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    frame_idx += 1
    print(f"Processing frame {frame_idx}")
    orig_shape = (frame.shape[1], frame.shape[0])
    
    if frame_idx < 5:
        print(f"Skipping frame {frame_idx}")
        continue
    
    if frame_idx == 5:
        cv2.imwrite("/kaggle/working/first_frame_front.jpg", frame)
        print("✅ Saved first frame for front to /kaggle/working/first_frame_front.jpg")
        print(f"Frame size: {frame.shape[1]}x{frame.shape[0]}")
    
    # Detect lanes (UFLD)
    with torch.no_grad():
        frame_with_lanes, fps = lane_detector.detect_lanes(frame)
    
    if frame_with_lanes is None:
        continue
    
    # Extract lane centers
    all_lanes = lane_detector.lanes_points[:4]
    valid_lanes = [lane for lane in all_lanes if len(lane) > 0]
    
    if len(valid_lanes) >= 2:
        lane_xs = sorted([np.median([pt[0] for pt in lane]) for lane in valid_lanes])[::-1]
    else:
        lane_xs = []
    
    # Depth estimation (simplified)
    model_shape = (1280, 720)
    depth_map = np.ones((model_shape[1], model_shape[0]), dtype=np.float32) * 10.0
    
    # Simulate the three views by transforming lane points
    views = [
        ("front", 0),
        ("front_left", offset_front_left),
        ("front_right", offset_front_right)
    ]
    
    all_lane_points_3d = []
    for view_name, x_offset in views:
        lane_points_3d = compute_lane_points_3d(all_lanes, depth_map, focal_length, cx, cy, orig_shape, model_shape, x_offset)
        all_lane_points_3d.append(lane_points_3d)
        print(f"Lane points 3D ({view_name}): {[len(lane) for lane in lane_points_3d]}")
    
    # YOLOv11 Detection
    yolo_results = yolo_model(frame)[0]
    detections = []
    tracked_yolo_boxes = []
    
    for det in yolo_results.boxes:
        x1, y1, x2, y2 = map(int, det.xyxy[0].tolist())
        conf = float(det.conf[0])
        cls_id = int(det.cls[0])
        if conf < 0.6 or cls_id not in CLASS_COLORS:
            continue
        w, h = x2 - x1, y2 - y1
        detections.append(([x1, y1, w, h], conf, cls_id))
        tracked_yolo_boxes.append({
            "bbox": [x1, y1, w, h],
            "cls_id": cls_id,
            "center": (x1 + w // 2, y1 + h // 2)
        })
    
    tracks = deep_sort.update_tracks(detections, frame=frame)
    
    detected_objects_front = []
    for track in tracks:
        if not track.is_confirmed():
            continue
        
        track_id = track.track_id
        x1, y1, x2, y2 = track.to_ltrb()
        w, h = x2 - x1, y2 - y1
        cx = int(x1 + w / 2)
        
        # Match track to YOLO box
        matched = None
        for yolo_box in tracked_yolo_boxes:
            yolo_cx, yolo_cy = yolo_box["center"]
            if abs(cx - yolo_cx) < 30:
                matched = yolo_box
                break
        
        if not matched:
            continue
        
        x, y, w, h = matched["bbox"]
        cls_id = matched["cls_id"]
        if cls_id not in SHOW_INFO_CLASSES:
            continue
        
        distance = estimate_distance([x, y, w, h])
        lane_number = assign_lane(x + w // 2, lane_xs)
        color = CLASS_COLORS[cls_id]
        
        # Compute 3D position
        bbox_area = w * h
        z_est = 20.0 * (10000 / (bbox_area + 1000))
        obj_center = np.array([x / 100.0, y / 100.0, z_est])
        
        detected_objects_front.append({
            "type": CLASS_NAMES[cls_id],
            "distance": distance,
            "center": obj_center,
            "color": color,
            "lane_idx": lane_number,
            "confidence": float(track.confidence) if hasattr(track, 'confidence') else 0.6
        })
        
        # Draw box and labels
        cv2.rectangle(frame_with_lanes, (x, y), (x + w, y + h), color, 2)
        cv2.putText(frame_with_lanes, f"{distance:.1f}m", (x + 5, y + h - 12),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)
        lane_text = f"Lane {lane_number}"
        text_size = cv2.getTextSize(lane_text, cv2.FONT_HERSHEY_SIMPLEX, 0.6, 2)[0]
        cv2.putText(frame_with_lanes, lane_text, (x + w - text_size[0] - 5, y + h - 12),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)
    
    # Simulate objects for front left and front right
    all_detected_objects = []
    for view_name, x_offset in views:
        for obj in detected_objects_front:
            obj_copy = obj.copy()
            obj_copy["center"] = transform_object_position(obj["center"], view_name, x_offset, fov=fov_simulated)
            all_detected_objects.append(obj_copy)
    
    # Add dummy object if none detected
    if not all_detected_objects:
        print("⚠ No objects detected, adding a dummy object for testing")
        all_detected_objects.append({
            "type": "debug_car",
            "center": np.array([0, 0, 10]),
            "color": (255, 0, 0),
            "lane_idx": "1",
            "confidence": 1.0,
            "distance": 10.0
        })
    
    # Update lane_detector.detected_objects for FCWS
    if lane_detector:
        lane_detector.detected_objects = all_detected_objects
    
    # Create BEV
    mean_img, dist_img = create_birds_eye_view(all_detected_objects, all_lane_points_3d, frame_idx=frame_idx)
    
    # Write to video outputs
    if frame_with_lanes.shape[:2] != (frame_height, frame_width):
        frame_with_lanes = cv2.resize(frame_with_lanes, (frame_width, frame_height))
    out.write(frame_with_lanes)
    mean_out.write(cv2.cvtColor(mean_img, cv2.COLOR_RGB2BGR))
    dist_out.write(cv2.cvtColor(dist_img, cv2.COLOR_RGB2BGR))
    
    # Save BEV points
    bev_points = []
    for obj in all_detected_objects:
        x, y = obj["center"][:2]
        bev_points.append({
            "frame": frame_idx,
            "x": x,
            "y": y,
            "source": "object",
            "lane_idx": obj["lane_idx"]
        })
    for lane_points_3d in all_lane_points_3d:
        for i, lane in enumerate(lane_points_3d):
            if len(lane) > 0:
                lane_center = np.mean(lane, axis=0)[:2]
                bev_points.append({
                    "frame": frame_idx,
                    "x": lane_center[0],
                    "y": lane_center[1],
                    "source": "lane",
                    "lane_idx": i
                })
    bev_points_data.extend(bev_points)

# Save BEV points to CSV
pd.DataFrame(bev_points_data).to_csv(bev_points_csv, index=False)
print(f"✅ BEV points saved to: {bev_points_csv}")

# Release resources
cap.release()
out.release()
mean_out.release()
dist_out.release()
cv2.destroyAllWindows()
print(f"✅ Processed videos saved to: {output_video_path}, {mean_future_output_path}, and {dist_future_output_path}")

# Print sample BEV points
print("Sample BEV Points from Last Frame:")
for pt_data in bev_points_data[-10:]:
    print(f"Frame {pt_data['frame']}: ({pt_data['x']:.2f}, {pt_data['y']:.2f}) from {pt_data['source']} (Lane {pt_data['lane_idx']})")